欢迎来到 YOLO 复现体验项目！！！让我们一起参考[《论文复现指南-cv版》](https://github.com/PaddlePaddle/models/blob/tipc/docs/lwfx/ArticleReproduction_CV.md)，按照步骤一步步完成[第四期飞桨论文复现赛](https://aistudio.baidu.com/aistudio/competition/detail/106/0/task-definition)真题：[You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640)，完整体验一篇经典论文的复现工作吧~

本项目中，为了体验完整的论文复现流程，在每个重要的打卡点均设置了一些代码缺失，你需要参考[官方实现](https://github.com/yjh0410/new-YOLOv1_PyTorch)补齐代码，并运行结果对比工具 `reprod_log`，验证各打卡点是否通过。每个打卡点通过计 20 分，满分 100 分。准备好了吗？让我们一起开启有趣的 coding 练习吧~

（注：如果你想复现其他方向的论文，欢迎参考[《论文复现指南-nlp版》](https://github.com/PaddlePaddle/models/blob/release/2.2/docs/lwfx/ArticleReproduction_NLP.md)、[《论文复现指南-Rec版》](https://github.com/PaddlePaddle/models/blob/release/2.2/docs/lwfx/ArticleReproduction_REC.md)）

## 配置环境
首先你需要安装论文复现赛中辅助自查和核验的工具 [reprod_log](https://github.com/WenmuZhou/reprod_log)，主要功能如下：
- 存取指定节点的输入输出 `tensor`
- 基于文件的 `tensor` 读写
- 2个字典的对比验证
- 对比结果的输出与记录

（注：本项目中使用的 `reprod_log` 工具已随项目打包并适配了 aistudio，请参考以下操作安装，无需从 github 下载）

In [ ]:
# 安装对比工具reprod_log
%cd /home/aistudio/reprod_log
!python setup.py bdist_wheel
!pip install dist/reprod_log-1.0.0-py3-none-any.whl --force-reinstall

/home/aistudio/reprod_log
running bdist_wheel
running build
running build_py
installing to build/bdist.linux-x86_64/wheel
running install
running install_lib
creating build/bdist.linux-x86_64/wheel
creating build/bdist.linux-x86_64/wheel/reprod_log
copying build/lib/reprod_log/utils.py -> build/bdist.linux-x86_64/wheel/reprod_log
copying build/lib/reprod_log/__init__.py -> build/bdist.linux-x86_64/wheel/reprod_log
copying build/lib/reprod_log/compare.py -> build/bdist.linux-x86_64/wheel/reprod_log
copying build/lib/reprod_log/ReprodDiffHelper.py -> build/bdist.linux-x86_64/wheel/reprod_log
copying build/lib/reprod_log/ReprodLogger.py -> build/bdist.linux-x86_64/wheel/reprod_log
running install_egg_info
running egg_info
writing reprod_log.egg-info/PKG-INFO
writing dependency_links to reprod_log.egg-info/dependency_links.txt
writing top-level names to reprod_log.egg-info/top_level.txt
reading manifest file 'reprod_log.egg-info/SOURCES.txt'
writing manifest file 'reprod_log.egg-info/SOURC

## 论文介绍

复现代码之前，先来认识一下我们要复现的经典检测模型 YOLO，这是一个端到端的目标检测算法，与之前的多阶段目标检测算法不同，YOLO 只需要进行一次 CNN 网络计算即可得到预测结果，这使得 YOLO 模型成为了业界公认的高精度、高效率、高实用性的模型。

<div align='left'>
  <img src='https://ai-studio-static-online.cdn.bcebos.com/a0b2ba74d09b46659ee2ee2107d06be9b190f87eeb96459ca8c64bd93c4fd468' width='600'/>
</div>

目前 YOLO 模型已经得到了学术界产业界的一致认可，并衍生出一系列的优质模型，其中包括 PaddleDetection 的明星模型 [PP-YOLO 和 PP-YOLOv2](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.3/configs/ppyolo/README.md)，在速度和精度上甚至超过了官方团队新出的 YOLOv5。

<div align='left'>
  <img src='https://ai-studio-static-online.cdn.bcebos.com/72b419d96b4a441283693354075e7a6d51d124dffd134e42bb7d4a74dc074088' width='300'/>
   <img src='https://ai-studio-static-online.cdn.bcebos.com/eb5d319ec175401783a92a8dd2b5ab27f525d8e962514a8e90c34fe1c6948a16' width='300'/>
</div>

YOLO 是如何实现"只用看一次"就得到目标检测框和物体分类结果的呢？整体来看，YOLO 算法采用一个单独的 CNN 模型实现端到端的目标检测，整个系统如下图所示：
- 首先将输入图片 resize 到 448 x 448，送入 CNN 网络；
- 之后网络将输入的图片分割成 S × S 网格，分别送到两条分支；
- 上面的检测分支每个单元格负责去检测那些中心点落在该格子内的目标，例如图中狗的中心点落在第5行第2格，则这一格负责预测狗的检测框；下面的分类分支每个单元格需要预测格子中图像的分类，并给出置信度。
- 最后结合检测分支与分类分支的输出结果，计算得出最终的检测图。

<div align='left'>
  <img src='https://ai-studio-static-online.cdn.bcebos.com/450744c2723e443c9fb6f4e5cd26980965beda300a5442a5bdb130fa9dad1d63' width='600'/>
</div>


## 开始练习
面对一篇计算机视觉论文，复现该论文的整体流程如下图所示，总共包含11个步骤。为了高效复现论文，设置了5个打卡点，如图中黄色框所示。

<div align='left'>
  <img src='https://ai-studio-static-online.cdn.bcebos.com/6198b7f186454bdd82a39bb900ee544950e32ca7c13c4beba275a7f39048adea' width='800'/>
</div>

接下来，我们将参考上图复现流程，逐步对齐并通过打卡点，复现 YOLO 模型。

### 打卡点1：前向对齐（含Step1：模型结构对齐）
对齐模型结构时，一般有3个主要步骤：
- 网络结构代码转换
- 权重转换
- 模型组网正确性验证

在本项目中，已完成了权重转换与组网正确性验证部分的代码，你只需要在网络结构代码 `.YOLO_reprod/checkpoint_1/yolo_paddle/models/yolo.py` 中，补齐 `forward` 部分的代码缺失，并运行以下指令测试参考模型与自己的模型的一次前向计算输出误差，并打印 log，测试打卡点1是否对齐。
```
cd ~/YOLO_reprod/checkpoint_1/yolo_paddle/
python forward_yolo.py

cd..
python checkpoint_1.py
```

注：
- 参考模型的网络结构代码放在 `~/YOLO_reprod/checkpoint_1/yolo_torch/models/yolo.py`，你可以参考这部分代码转写你的 paddle 模型代码；
- 权重转换代码放在 `~/YOLO_reprod/weights_trans/torch2paddle.py`，有兴趣可以自行研读，模型转换的注意事项详见[《论文复现手册》](https://github.com/PaddlePaddle/models/blob/release/2.2/docs/lwfx/ArticleReproduction_CV.md#312-%E6%9D%83%E9%87%8D%E8%BD%AC%E6%8D%A2)。

In [ ]:
# 下载官方实现模型权重，转换为paddle模型权重(此步骤跳过，转换后模型已存储在 ~/YOLO_reprod/weights_trans/yolo_paddle.pdparams)
# %cd /home/aistudio/YOLO_reprod/weights_trans/
# !python torch2paddle.py

# 生成假数据(此步骤跳过，假数据与假标签已存储在 ~/YOLO_reprod/fake_data/)
# %cd /home/aistudio/YOLO_reprod/fake_data/
# !python gen_fake_data.py

In [ ]:
# 打卡点1对齐（对齐网络结构）
# 生成paddle的前向数据
%cd /home/aistudio/YOLO_reprod/checkpoint_1/yolo_paddle/
!python forward_yolo.py

# 与官方实现模型的前向数据对比，生成log
%cd ..
!python checkpoint_1.py

/home/aistudio/YOLO_reprod/checkpoint_1/yolo_paddle
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/aistudio/YOLO_reprod/checkpoint_1
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[2022/01/10 10:26:10] root INFO: logits: 
[2022/01/10 10:26:10] root INFO: 	mean diff: check passed: True, value: 1.155436994173588e-07
[2022/01/10 10:26:10] root INFO: diff check passed


恭喜你对齐了打卡点1：前向对齐！！！接下来要完成打卡点2：评估对齐，这一步包含两个步骤，你需要读取测试数据后，使用转换后模型进行评估，并实现评估指标对齐。

### 打卡点2：评估对齐（含Step 2：验证/测试集数据读取对齐、Step 3：评估指标对齐）
对齐评估指标，你需要完成Step 2：验证/测试集数据读取对齐 和 Step 3：评估指标对齐，分别会打印两个对齐 log。

首先是 **Step 2：验证/测试集数据读取对齐**，在这一步，我们需要
* 将 aistudio 挂载的数据集 VOCdevkit 解压；
* 参考官方实现代码转写 paddle 数据加载代码；
* 固定预处理方式，在 `~/YOLO_reprod/checkpoint_2/test_vocdata.py` 中定义 `dataset` 与 `dataloader` ，对比输出误差，打印 log。

注：由于 `dataset` 与 `dataloader` 输出对齐中涉及的数据量较大，无法打包保存在项目中，**此步骤跳过**。

In [ ]:
# dataset与dataloader输出对齐(涉及数据量较大，不便于在aistudio操作，此步跳过)
%cd /home/aistudio/YOLO_reprod/checkpoint_2/
!python test_vocdata.py

/home/aistudio/YOLO_reprod/checkpoint_2
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Traceback (most recent call last):
  File "test_vocdata.py", line 86, in <module>
    test_data_pipeline()
  File "test_vocdata.py", line 58, in test_data_pipeline
    paddle_dataset, paddle_dataloader = build_paddle_data_pipeline()
  File "test_vocdata.py", line 42, in build_paddle_data_pipeline
    transform=BaseTransform((train_size), (0, 0, 0))
  File "/home/aistudio/YOLO_reprod/checkpoint_2/yolo_paddle/data/voc0712.py", line 117, in __init__
    for line in open(osp.join(rootpath, 'ImageSets', 'Main', name + '.txt')):
FileNotFoundError: [Errno 2] No such file or directory: '/home/bml/.storage/mnt/v-uimvattx3skxxwr7/org/workflow/VOCdevkit/VOC2007/ImageSets/Main/trainval.txt'


接下来是 **Step 3：评估指标对齐**，在这一步，我们需要：
* 将测试集数据载入网络，定义模型的 `eval` 部分代码；
* 打印评估结果并将评估指标 `mAP` 保存；
* 评估指标正确性验证。

在本项目中，你需要在 `~/YOLO_reprod/checkpoint_2/yolo_paddle/metric_eval.py` 中完成 `voc_test` 部分的代码缺失，并运行以下指令测试参考代码与自己的 `eval` 代码的评估指标输出误差，并打印 log，测试打卡点2是否对齐。
```
cd ~/YOLO_reprod/checkpoint_2/yolo_paddle/
sh run_eval.sh

cd ..
python checkpoint_2.py
```




In [ ]:
# 解压数据集
%cd /home/aistudio/
!tar -xf /home/aistudio/data/data9837/VOCtrainval_06-Nov-2007.tar
!tar -xf /home/aistudio/data/data9837/VOCtest_06-Nov-2007.tar
!tar -xf /home/aistudio/data/data9837/VOCtrainval_11-May-2012.tar

/home/aistudio


In [ ]:
# 打卡点2对齐（对齐评估指标）
# 评估paddle模型，生成评估指标map
%cd /home/aistudio/YOLO_reprod/checkpoint_2/yolo_paddle/
!sh run_eval.sh
    
# 与官方实现模型的评估指标数据对比，生成log
%cd ..
!python checkpoint_2.py

/home/aistudio/YOLO_reprod/checkpoint_2/yolo_paddle
eval on voc ...
use gpu
W0110 10:39:58.974001   376 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0110 10:39:58.977844   376 device_context.cc:465] device: 0, cuDNN Version: 7.6.


恭喜你对齐了打卡点2：评估对齐！！！接下来要完成打卡点3：损失函数对齐。

### 打卡点3：损失函数对齐（含 Step 4:损失函数对齐）
对齐损失函数，你需要参考以下步骤：
* 定义模型训练代码，加载权重与假数据假标签，获取一次前向计算的 loss 结果并保存；
* 损失函数正确性验证。

在本项目中，你需要在 `~/YOLO_reprod/checkpoint_3/yolo_paddle/tools.py` 中完成 `loss` 部分的代码缺失，并运行以下指令测试参考代码与自己代码的一次前向训练输出误差，并打印 log，测试打卡点3是否对齐。
```
cd ~/YOLO_reprod/checkpoint_3/yolo_paddle/
python loss_train.py

cd ..
python checkpoint_3.py
```
注：
`CrossEntropyLoss`有一定的区别需要注意：PaddlePaddle 提供了对软标签、指定 softmax 计算纬度的支持。即 `paddle.nn.CrossEntropyLoss` 默认是在最后一维(axis=-1)计算损失函数，而 `torch.nn.CrossEntropyLoss` 是在axis=1的地方计算损失函数，因此如果输入的维度大于 2，这里需要保证计算的维(axis)相同，否则可能会出错。

In [ ]:
# 打卡点3对齐（对齐损失函数）
# 使用假数据与假标签获取paddle模型的损失函数
%cd /home/aistudio/YOLO_reprod/checkpoint_3/yolo_paddle/
!python loss_train.py

# 与官方实现模型的损失函数数据对比，生成log，阈值改为1e-5
%cd ..
!python checkpoint_3.py

/home/aistudio/YOLO_reprod/checkpoint_3/yolo_paddle
/home/aistudio/YOLO_reprod/checkpoint_3
[2022/01/10 11:20:26] root INFO: conf_loss: 
[2022/01/10 11:20:26] root INFO: 	mean diff: check passed: True, value: 5.982089037459559e-07
[2022/01/10 11:20:26] root INFO: cls_loss: 
[2022/01/10 11:20:26] root INFO: 	mean diff: check passed: True, value: 7.152557373046875e-07
[2022/01/10 11:20:26] root INFO: txtytwth_loss: 
[2022/01/10 11:20:26] root INFO: 	mean diff: check passed: True, value: 4.291534423828125e-06
[2022/01/10 11:20:26] root INFO: total_loss: 
[2022/01/10 11:20:26] root INFO: 	mean diff: check passed: True, value: 3.2164883627672225e-06
[2022/01/10 11:20:26] root INFO: diff check passed


恭喜你对齐了打卡点3：损失函数对齐！！！接下来要完成打卡点4：反向对齐。本步骤包含：优化器对齐、学习率对齐、正则化策略对齐和反向对齐。其中，正则化策略在 YOLO 中是写死的，因此该步骤跳过。我们可以划分为两大步骤完成：优化器学习率对齐和反向对齐。

### 打卡点4：反向对齐（含 Step 5：优化器对齐、Step 6：学习率对齐、Step 7：正则化策略对齐、Step 8：反向对齐）

对齐反向计算，你需要完成Step 5+6：优化器学习率对齐 和 Step 8：反向对齐，分别会打印两个对齐 log。每个 log 计 10 分。

首先是 **Step 5+6：优化器学习率对齐**，在这一步，我们需要
* 定义优化器
* 定义学习率规则
* 设置 90 个 epoch，将每个 epoch 的学习率保存
* 比对学习率正确性

在本项目中，你需要在 `~/YOLO_reprod/checkpoint_4/yolo_paddle/lr_optim_eval.py` 中完成 **优化器与学习率** 部分的代码缺失，并运行以下指令测试参考代码与自己代码的若干次学习率输出误差，并打印 log。
```
cd ~/YOLO_reprod/checkpoint_4/yolo_paddle/
python lr_optim_eval.py

cd ..
python test_lr.py
```
注：
- PaddlePaddle 的 `SGD` 不支持动量更新、动量衰减和 `Nesterov` 动量，这里需要使用 1paddle.optimizer.Momentum` API实现这些功能。
- PaddlePaddle 定义的优化器支持 `optimizer.set_lr()` 方式直接修改学习率，而不需要额外定义 `set_lr` 的函数。



In [ ]:
# 优化器与学习率对齐
%cd /home/aistudio/YOLO_reprod/checkpoint_4/yolo_paddle/
!python lr_optim_eval.py

%cd ..
!python test_lr.py

/home/aistudio/YOLO_reprod/checkpoint_4/yolo_paddle
/home/aistudio/YOLO_reprod/checkpoint_4
[2022/01/10 12:37:02] root INFO: lr: 
[2022/01/10 12:37:02] root INFO: 	mean diff: check passed: True, value: 0.0
[2022/01/10 12:37:02] root INFO: diff check passed


接下来是 **Step 8：反向对齐**，此处推荐使用假数据与假标签，实现可复现的结果。在这一步，我们需要
* 检查两个代码的训练超参数全部一致，如优化器及其超参数、学习率、BatchNorm/LayerNorm中的eps等；
* 将 PaddlePaddle 与 PyTorch 网络中涉及的所有随机操作全部关闭，如 `dropout`、`drop_path` 等，推荐将模型设置为 `eval` 模式（ `model.eval()` ）；
* 加载相同的模型权重，将准备好的数据分别传入网络并迭代，观察二者loss是否一致（此处 `batch_size` 要一致，如果使用多个真实数据，要保证传入网络的顺序一致）；
* 如果经过2轮以上，loss 均可以对齐，则基本可以认为反向对齐。

在本项目中，你需要把上一步中定义的优化器放置在反向对齐代码 `~/YOLO_reprod/checkpoint_4/yolo_paddle/bp_train.py` 中，并运行以下指令测试参考代码与自己代码的 2 次以上损失函数输出误差，并打印 log，测试打卡点4是否对齐。
```
cd ~/YOLO_reprod/checkpoint_4/yolo_paddle/
python bp_train.py

cd ..
python checkpoint_4.py
```

In [ ]:
# 打卡点4对齐（对齐反向传播）
# 使用假数据与假标签获取paddle模型的5个epoch反向损失函数
%cd /home/aistudio/YOLO_reprod/checkpoint_4/yolo_paddle/
!python bp_train.py

# 与官方实现模型的反向传播数据对比，生成log，阈值改为1e-5
%cd ..
!python checkpoint_4.py

/home/aistudio/YOLO_reprod/checkpoint_4/yolo_paddle
W0110 16:45:33.614033  1644 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0110 16:45:33.618849  1644 device_context.cc:465] device: 0, cuDNN Version: 7.6.
/home/aistudio/YOLO_reprod/checkpoint_4
[2022/01/10 16:45:40] root INFO: conf_loss_0: 
[2022/01/10 16:45:40] root INFO: 	mean diff: check passed: True, value: 6.395274043669019e-08
[2022/01/10 16:45:40] root INFO: conf_loss_1: 
[2022/01/10 16:45:40] root INFO: 	mean diff: check passed: True, value: 2.4378674656766464e-06
[2022/01/10 16:45:40] root INFO: conf_loss_2: 
[2022/01/10 16:45:40] root INFO: 	mean diff: check passed: True, value: 9.078661482142536e-06
[2022/01/10 16:45:40] root INFO: conf_loss_3: 
[2022/01/10 16:45:40] root INFO: 	mean diff: check passed: False, value: 2.107878318646783e-05
[2022/01/10 16:45:40] root INFO: conf_loss_4: 
[2022/01/10 16:45:40] root INFO: 	mean diff: check passed

恭喜你对齐了打卡点4：反向对齐！！！接下来要完成打卡点5：精度对齐。训练集和测试集数据读取方式相似，因此 Step9 可以跳过；conv+bn 结构的 CNN 网络通常对网络初始化并不敏感，因此选择随机初始化即可， Step10 也可以跳过；本节我们重点调整 Step11：训练对齐。

### 打卡点5：精度对齐（含 Step9：训练集数据读取对齐、Step10：网络初始化对齐、Step11：训练对齐）

终于到了最激动人心的训练环节，~~让我们一把梭哈，不行回家~~~经过前4步的对齐，通常来讲训练精度不会有特别大的误差，我们定义好训练代码 `train.py` 后，就可以开启漫长的深度学习训练过程了。

注：由于 YOLO 训练时间过久，有兴趣的同学可以尝试训练一下，若你时间有限，也可以加载 `YOLO_reprod/checkpoint_5/yolo_paddle/yolo-model.pdparams` 权重参数，进行后续的打卡。

In [6]:
# 开始训练，并将最佳模型参数保存
%cd /home/aistudio/YOLO_reprod/checkpoint_5/yolo_paddle
!sh train.sh

/home/aistudio/YOLO_reprod/checkpoint_5/yolo_paddle
W0110 16:48:49.422663  2017 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0110 16:48:49.427881  2017 device_context.cc:465] device: 0, cuDNN Version: 7.6.
im_detect: 1/4952 0.025s
im_detect: 501/4952 0.020s
im_detect: 1001/4952 0.021s
im_detect: 1501/4952 0.020s
im_detect: 2001/4952 0.021s
im_detect: 2501/4952 0.021s
im_detect: 3001/4952 0.019s
im_detect: 3501/4952 0.019s
im_detect: 4001/4952 0.029s
im_detect: 4501/4952 0.022s
Evaluating detections
VOC07 metric? Yes
AP for aeroplane = 0.0000
AP for bicycle = 0.0007
AP for bird = 0.0003
AP for boat = 0.0000
AP for bottle = 0.0001
AP for bus = 0.0000
AP for car = 0.0057
AP for cat = 0.0041
AP for chair = 0.0006
AP for cow = 0.0000
AP for diningtable = 0.0000
AP for dog = 0.0015
AP for horse = 0.0002
AP for motorbike = 0.0002
AP for person = 0.0078
AP for pottedplant = 0.0000
AP for sheep = 0.0000
AP for s

In [7]:
# 测试训练最佳模型在测试集上的指标并保存
%cd /home/aistudio/YOLO_reprod/checkpoint_5/yolo_paddle/
!sh eval.sh

/home/aistudio/YOLO_reprod/checkpoint_5/yolo_paddle
eval on voc ...
use gpu
W0110 18:17:41.922364 10978 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.0, Runtime API Version: 10.1
W0110 18:17:41.927183 10978 device_context.cc:465] device: 0, cuDNN Version: 7.6.
Finished loading model!
im_detect: 1/4952 0.022s
im_detect: 501/4952 0.016s
im_detect: 1001/4952 0.015s
im_detect: 1501/4952 0.015s
im_detect: 2001/4952 0.018s
im_detect: 2501/4952 0.017s
im_detect: 3001/4952 0.016s
im_detect: 3501/4952 0.016s
im_detect: 4001/4952 0.016s
im_detect: 4501/4952 0.017s
Evaluating detections
Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC resu

In [8]:
# 对比精度，打印log，阈值调整为0.0015
%cd /home/aistudio/YOLO_reprod/checkpoint_5/
!python checkpoint_5.py

/home/aistudio/YOLO_reprod/checkpoint_5
[2022/01/10 18:19:59] root INFO: map: 
[2022/01/10 18:19:59] root INFO: 	mean diff: check passed: True, value: 0.0002689957618713379
[2022/01/10 18:19:59] root INFO: diff check passed


 **恭喜你完成了目标检测经典论文 YOLO 的代码复现！！！**  现在你已经是一个熟练的炼丹师了，快去开启你的深度学习之旅吧~你可以去 飞桨第五期论文复现挑战赛 选取自己感兴趣的论文并完成复现，会有丰厚奖金为你准备！你也可以选取一篇未列入复现赛的顶会论文自行复现，祝你好运哦~

注：若是复现过程中遇到困难，你可以在 github 给我们提 issue，并在标题前备注【论文复现】，会有百度研发工程师为你解答~
<div align='left'>
  <img src='https://ai-studio-static-online.cdn.bcebos.com/f956037477fe42a0bdb172387c8cc37095805a2911c54125a6f93849523ab3fb' width='300'/>
</div>
